# Label-Studio

Label-studio est un outil d'annotation de données. Il offre une large variété de tâches d'annotation pour la vision par ordinateur, le traitement de language naturel, traitement audio et l'analyse de séries chronologiques. 

Dans ce tutoriel, on explique comment connecter label-studio au serveur minIO de SSPCloud pour importer et exporter les données directement du stockage de Datalab. On va connecter un bucket qui contient d'images de différents plats qu'on veut classer en deux types: pizza et not-pizza

## 1- Configuration de l'annotation

En créant un nouveau projet sur Label-Studio, on peut consulter les différentes configurations d'anntotation qu'il offre sous la rubrique ``labeling setup``. Dans notre cas, on est intéressé par la configuration de ``Image classification`` qui est une tâche de ``computer vision``

Dans l'espace ``add choices``, on peut ajouter nos classes qui sont pizza et not-pizza. 

**L'ordre de l'ajout de classes est important**. Dans notre cas, l'indice 0 correspond à la classe ``pizza`` et l'indice 1 correspond à la classe ``not-pizza``.

<img src="notebook-img/create-project.jpg">

Dans la rubrique ``Data Import``, on peut importer des données locales ou télécharger une base de données en utilisant un url. Mais, dans ce tutoriel, on s'intéresse à l'importation de données d'un bucket s3 sur le SSPCloud. Donc on sauvergarde le projet ``save`` et on passe à l'étape suivante.

## 2- Connection sur s3 de SSPCloud

La plupart de services de DataLab utilisent le stockage s3 de SSPCloud. Donc il est fortement recommandé d'importer vos données sur votre espace au début de votre projet une fois pour toute. 

On ouvre le projet créé précédemment sur Label-Studio et on passe à la rubrique ``settings -> CloudStorage -> Add Source Storage`` qui permet d'importer les données à annoter d'un bucket s3 sur SSPCloud. Ici, il suffit de configurer le champ ``Bucket Name`` qui indique le nom de Bucket qui contient les données. Le nom de votre Bucket personnel est disponible dans la rubrique ``My Files`` sur Datalab.

<img src="notebook-img/settings.jpg">

In [5]:
! mc ls s3/mbenxsalha/diffusion/pizza-not-pizza

[2022-10-10 13:43:05 UTC]    39B STANDARD .keep
[2022-10-20 08:02:31 UTC]     0B not_pizza/
[2022-10-20 08:02:31 UTC]     0B pizza/


Le bucket ``mbenxsalha`` contient les données qu'on va utiliser dans ce tutoriel sous la préfixe``diffusion/pizza-not-pizza`` . Cette préfixe contient deux dossiers ``pizza`` et ``not_pizza`` contenant chacun des données. Pour pouvoir lire des données dans des multiples dossiers sous une préfixe, on doit activer l'option ``Recursive scan``. De plus, pour lire les fichiers de type autre que JSON, on doit activer l'option ``Treat evert bucket object as a source file``. Les autres paramètres d'accés (token, key, endpoint, region) sont configurés en avance sur label-studio avec des variables d'environnement, donc on peut les laisser vides. On peut tester notre conenction en cliquant sur ``test`` et d'ajouter le stockage avec ``add storage``. Enfin, pour ajouter les données sur ce stockage au projet d'annotation, on clique sur ``sync storage``.

Dans settings, on peut aussi configurer un ``target cloud storage`` en suivant les même étapes. Cela permet d'exporter les annotations créées dans le projet vers un bucket s3.

<img src="notebook-img/label.jpg">

## 3- Connecter un modèle à Label-Studio

In [8]:
#install dependencies
!pip install label-studio label-studio-ml
!pip install torch torchvision
!pip install Flask==2.1.0

Label-studio offre la possibilité de se connecter à un modèle. Cela permet de :
- pré-annoter les données en utilisant un modèle d'inférence
- entrainer un modèle en utilisant les données annotés sur Label-Studio


Pour ce faire, il faut exposer le modèle avec une API en utilisant le SDK ``label-studio-ml``. L'api doit être une sous-classe de ``label_studio_ml.model.LabelStudioMLBase``. Ainsi, l'api hérite deux fonctions abstraites ``fit`` et ``predict`` qui permettent d'entrainer et d'inférer le modèle. Le code de l'api utilisée ici est disponible dans le fichier ``label_studio_server.py``.

On exécute la commande suivante:

In [ ]:
!label-studio-ml init tutorial --script label_studio_server.py

Un dossier contenant les fichiers nécessaires à l'exécution de l'api vient d'être créer. 

Avant d'activer le serveur, il faut que le paramètre ``security.allowlist.enabled``de jupyter soit désactivée.
<img src="notebook-img/security.jpg">


Ensuite, on doit exposer l'api au monde extérieur pour pouvoir y connecter. Donc, dans la configuration de Jupyter, on active ``Networking -> enable a custom user port``. L'api créée par Label-studio utilise par défault le port 9090.
<img src="notebook-img/port.jpg">

**Tous ces paramètres sont préconfigurés dans ce tutoriel !**

Pour lancer l'API, on exécute la commande suivante:

In [ ]:
!label-studio-ml start tutorial

Vous pouvez trouver le lien de l'api que vous venez de créer en changeant le lien du notebook comme dans l'exemple suivant:
- lien de notebook: https://user-username-239011-0.user.lab.sspcloud.fr/
- modifier le "-0" par "-user". Donc pour cet exemple, le lien de l'API est: https://user-username-239011-user.user.lab.sspcloud.fr/


Enfin, vous pouvez connecter à l'API sur Label-Studio en utilisant ce lien dans ``Settings -> Machine Learning -> Add Model -> URL``.
Pour inférer le modèle, il faut activer l'option ``use for active preannotations``
<img src="notebook-img/ml.jpg">